In [1]:
import sys
import os
sys.path.append(os.path.abspath('../timecorr/'))

import matplotlib.pyplot as plt
from math import log
import numpy as np
from scipy.linalg import cholesky
from scipy.spatial.distance import squareform
from scipy.stats.stats import pearsonr


block_length = 1
covariance_num = 300
time_len = block_length * covariance_num
activation_num = 500
activations = np.random.normal(0,10,[activation_num, time_len])
correlations = np.zeros([covariance_num,activation_num,activation_num])
correlation1,correlation2 = np.zeros([activation_num,activation_num]), np.zeros([activation_num,activation_num])

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def cholesky_ramp_correlation_data():
    global activations, correlations,correlation1,correlation2
    correlations = np.zeros([covariance_num,activation_num,activation_num])
    activations = np.random.normal(0,10,[activation_num, time_len])
    
    while not is_pos_def(correlation1):
        feature_map = np.random.normal(0,1,[activation_num,activation_num])
        correlation1 = np.dot(feature_map,feature_map.T)
        correlation1 = correlation1/np.max(abs(correlation1))
    while not is_pos_def(correlation2):
        feature_map1 = np.random.normal(0,1,[activation_num,activation_num])
        correlation2 = np.dot(feature_map1,feature_map1.T)
        correlation2 = correlation2/np.max(abs(correlation2))
    for i in range(time_len):
        cov_temp = (1000-i)*correlation1/float(time_len)+i*correlation2/float(time_len)
        correlations[i] = cov_temp/np.max(abs(cov_temp))
        activations[:, i] = np.dot(cholesky(correlations[i]),activations[:,i])


cholesky_ramp_correlation_data()


In [2]:
%%time
from _shared.helpers_temp1 import wcorr as lolol
variance = 1000
repetitions=100
timecorr_recovery = np.zeros([time_len,(activation_num * (activation_num-1) / 2)])
for i in range(repetitions):
    timecorr_recovery = lolol(activations,variance)

KeyboardInterrupt: 

In [4]:
%%time
from _shared.helpers import wcorr as haha
variance = 1000
repetitions=1
timecorr_recovery = np.zeros([time_len,(activation_num * (activation_num-1) / 2)])
for i in range(repetitions):
    timecorr_recovery = haha(activations,variance)

CPU times: user 3min 36s, sys: 2.92 s, total: 3min 39s
Wall time: 3min 41s
